# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
conn.set_session(autocommit=True)

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
# how many file ?
len(song_files)

82

In [6]:
filepath = song_files[0]

In [7]:
import json
from pprint import pprint
with open(filepath, 'r') as f:
    song_data = json.load(f)
pprint(song_data)

{'artist_id': 'ARD7TVE1187B99BFB1',
 'artist_latitude': None,
 'artist_location': 'California - LA',
 'artist_longitude': None,
 'artist_name': 'Casual',
 'duration': 218.93179,
 'num_songs': 1,
 'song_id': 'SOMZWCG12A8C13C480',
 'title': "I Didn't Mean To",
 'year': 0}


In [8]:
# create df to contain all song data from all json filesa
df_song_data = pd.DataFrame()
df_song_data = df_song_data.append(pd.read_json(filepath, typ='series'),ignore_index=True)

In [9]:
df_song_data.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,None,California - LA,None,Casual,218.93179,1.0,SOMZWCG12A8C13C480,I Didn't Mean To,0.0


In [10]:
# what's the shape of my df ?
df_song_data.shape

(1, 10)

In [11]:
# df columns
df_song_data.columns

Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
      dtype='object')

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [12]:
song_data = df_song_data.iloc[0][['song_id', 'title', 'artist_id', 'year', 'duration']]
song_data = list(song_data.values)
song_data

['SOMZWCG12A8C13C480',
 "I Didn't Mean To",
 'ARD7TVE1187B99BFB1',
 0.0,
 218.93179000000001]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [13]:
try :
    cur.execute(song_table_insert, song_data)
except psycopg2.Error as e:
        cur.execute("rollback;")

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [14]:
df_song_data.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,None,California - LA,None,Casual,218.93179,1.0,SOMZWCG12A8C13C480,I Didn't Mean To,0.0


In [15]:
artist_data = df_song_data.iloc[0][['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_data = list(artist_data.values)
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [16]:
try :
    cur.execute(artist_table_insert, artist_data)
except psycopg2.Error as e:
    cur.execute("rollback;")

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [17]:
log_files = get_files('data/log_data')

In [18]:
len(log_files)

31

In [19]:
filepath = log_files[0]

In [20]:
df_log_data = pd.read_json(filepath, lines=True)

In [21]:
df_log_data.shape

(457, 18)

In [22]:
df_log_data.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Grass Roots,Logged In,Sara,F,72,Johnson,166.71302,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Let's Live For Today,200,1542153802796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
1,Stars,Logged In,Sara,F,73,Johnson,298.94485,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Time Can Never Kill The True Heart,200,1542153968796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95


In [23]:
df_log_data.columns.values

array(['artist', 'auth', 'firstName', 'gender', 'itemInSession',
       'lastName', 'length', 'level', 'location', 'method', 'page',
       'registration', 'sessionId', 'song', 'status', 'ts', 'userAgent',
       'userId'], dtype=object)

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [24]:
pd.set_option('mode.chained_assignment', None)

In [25]:
df = df_log_data.loc[df_log_data['page']=='NextSong']

In [26]:
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Grass Roots,Logged In,Sara,F,72,Johnson,166.71302,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Let's Live For Today,200,1542153802796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
1,Stars,Logged In,Sara,F,73,Johnson,298.94485,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Time Can Never Kill The True Heart,200,1542153968796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95


In [27]:
df.shape

(390, 18)

In [28]:
# time df
df_t = df[['ts']]

In [29]:
df_t['datetime'] = pd.to_datetime(df_t['ts'], unit='ms')

In [30]:
df_t['hour'] = df_t['datetime'].dt.hour
df_t['day'] = df_t['datetime'].dt.day
df_t['week'] = df_t['datetime'].dt.week
df_t['month'] = df_t['datetime'].dt.month
df_t['year'] = df_t['datetime'].dt.year
df_t['weekday'] = df_t['datetime'].dt.weekday

In [31]:
df_t.head()

,ts,datetime,hour,day,week,month,year,weekday
0,1542153802796,2018-11-14 00:03:22.796,0,14,46,11,2018,2
1,1542153968796,2018-11-14 00:06:08.796,0,14,46,11,2018,2
2,1542154266796,2018-11-14 00:11:06.796,0,14,46,11,2018,2
3,1542154657796,2018-11-14 00:17:37.796,0,14,46,11,2018,2
4,1542154825796,2018-11-14 00:20:25.796,0,14,46,11,2018,2


In [32]:
column_labels = list(df_t.columns.values)
column_labels.remove('datetime')
column_labels

['ts', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [33]:
time_data = (list(df_t['ts'].values), 
             list(df_t['hour'].values), 
             list(df_t['day'].values), 
             list(df_t['week'].values), 
             list(df_t['month'].values), 
             list(df_t['year'].values),
             list(df_t['weekday'].values)
            )

column_labels = (column_labels)

In [34]:
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
time_df.head()

,ts,hour,day,week,month,year,weekday
0,1542153802796,0,14,46,11,2018,2
1,1542153968796,0,14,46,11,2018,2
2,1542154266796,0,14,46,11,2018,2
3,1542154657796,0,14,46,11,2018,2
4,1542154825796,0,14,46,11,2018,2


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [35]:
for i, row in time_df.iterrows():
    try :
        cur.execute(time_table_insert, list(row))
    except psycopg2.Error as e:
        cur.execute("rollback;")

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [36]:
df_log_data.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Grass Roots,Logged In,Sara,F,72,Johnson,166.71302,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Let's Live For Today,200,1542153802796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95
1,Stars,Logged In,Sara,F,73,Johnson,298.94485,paid,"Winston-Salem, NC",PUT,NextSong,1.540809e+12,411,Time Can Never Kill The True Heart,200,1542153968796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",95


In [37]:
user_df = df_log_data.loc[:, ['userId', 'firstName', 'lastName', 'gender', 'level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [38]:
user_df.head()

,userId,firstName,lastName,gender,level
0,95,Sara,Johnson,F,paid
1,95,Sara,Johnson,F,paid
2,95,Sara,Johnson,F,paid
3,95,Sara,Johnson,F,paid
4,95,Sara,Johnson,F,paid


In [39]:
for i, row in user_df.iterrows():
    try :
        cur.execute(user_table_insert, row)
    except psycopg2.Error as e:
        cur.execute("rollback;")

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [40]:
df_log_data.columns.values

array(['artist', 'auth', 'firstName', 'gender', 'itemInSession',
       'lastName', 'length', 'level', 'location', 'method', 'page',
       'registration', 'sessionId', 'song', 'status', 'ts', 'userAgent',
       'userId'], dtype=object)

In [41]:
df =  df_log_data.loc[:, ['ts', 'userId', 'level', 'sessionId', 'location', 'userAgent', 'song', 'artist', 'length']]

In [42]:
# start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    try :
        cur.execute(songplay_table_insert, songplay_data)
    except psycopg2.Error as e:
        cur.execute("rollback;")

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.